# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [12]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201213_201451_cgan_long_run_model4', '20201219_193…

In [13]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201220_073758_lambda2.0


In [14]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

## Plot Losses

In [15]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [16]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
124991,124991.0,39.0,0.103187,0.088260,0.191447,4.504684,157.179153,152.674469,-2.822345,169.305649,-2.974279,4.516505,-4.150256,-4.484783,0.127553
124992,124992.0,39.0,0.052276,0.103978,0.156254,5.707906,159.908005,154.200104,-2.897568,167.202972,-3.060237,4.849648,-3.783720,-5.702433,0.126592
124993,124993.0,39.0,0.111424,0.103923,0.215347,4.468016,153.507858,149.039841,-2.871263,159.711349,-2.960070,3.586196,-5.382584,-4.446530,0.127816
124994,124994.0,39.0,0.116637,0.073345,0.189982,4.972733,159.123596,154.150864,-2.740103,153.117661,-2.627275,4.813157,-3.976276,-4.959853,0.126689
124995,124995.0,39.0,0.071904,0.100926,0.172829,4.894724,157.993301,153.098572,-2.701926,158.289673,-2.882916,4.515562,-4.694671,-4.883068,0.126472
124996,124996.0,39.0,0.098563,0.080828,0.179392,4.857952,162.754105,157.896149,-2.683530,183.610611,-2.431377,5.059011,-4.430486,-4.844821,0.126726
124997,124997.0,39.0,0.088988,0.124457,0.213445,5.635336,160.069336,154.434006,-2.539092,161.565704,-2.963248,4.736764,-4.391994,-5.628266,0.126951
124998,124998.0,39.0,0.124430,0.085627,0.210057,4.544389,162.440552,157.896164,-2.666374,176.024979,-2.890784,4.070927,-4.923419,-4.526699,0.127561
124999,124999.0,39.0,0.085142,0.080086,0.165228,4.930931,161.579742,156.648804,-2.717851,166.305817,-2.648703,4.955451,-4.128489,-4.918892,0.126413
125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171.286880,-2.958859,NaN,NaN,NaN,NaN


In [17]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [18]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-3.091027262210846


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
87959,87959.0,28.0,0.069333,0.133022,0.202355,5.257186,162.274368,157.017181,-3.254523,173.289719,-3.262036,4.617714,-4.084970,-5.244422,0.128029
87958,87958.0,28.0,0.134411,0.133091,0.267502,4.251965,160.384155,156.132187,-3.212855,165.361053,-3.261466,4.490849,-3.131538,-4.227014,0.127338
87966,87966.0,28.0,0.123655,0.083543,0.207198,4.646482,165.050598,160.404114,-3.239736,170.698532,-3.252898,4.132565,-3.996469,-4.626503,0.127483
87952,87952.0,28.0,0.120951,0.162980,0.283931,5.668611,161.622330,155.953720,-3.163805,162.911972,-3.252406,5.135420,-2.723379,-5.661331,0.127256
105197,105197.0,33.0,0.094444,0.102566,0.197010,5.339356,155.679749,150.340393,-3.155228,178.069275,-3.241058,5.497356,-4.388076,-5.328547,0.128754
87944,87944.0,28.0,0.145730,0.110045,0.255774,3.086662,165.097870,162.011215,-3.075609,156.789337,-3.237451,2.950878,-3.670204,-2.990322,0.127450
87953,87953.0,28.0,0.108757,0.069656,0.178413,4.943049,160.020874,155.077820,-3.203660,152.532532,-3.237336,3.638846,-4.544709,-4.929323,0.126319
87941,87941.0,28.0,0.147713,0.122562,0.270275,4.133932,164.184692,160.050766,-2.455491,171.547394,-3.234285,3.101808,-4.035998,-4.099337,0.127956
121447,121447.0,38.0,0.115221,0.094295,0.209516,4.998542,158.946381,153.947845,-2.913225,167.428894,-3.230563,4.589681,-4.661078,-4.986238,0.126859
105200,105200.0,33.0,0.091813,0.096171,0.187984,5.357634,157.968475,152.610840,-2.991144,172.345673,-3.229772,4.660221,-3.895590,-5.348387,0.127723


In [19]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
87959,87959.0,28.0,0.069333,0.133022,0.202355,5.257186,162.274368,157.017181,-3.254523,173.289719,-3.262036,4.617714,-4.084970,-5.244422,0.128029
87958,87958.0,28.0,0.134411,0.133091,0.267502,4.251965,160.384155,156.132187,-3.212855,165.361053,-3.261466,4.490849,-3.131538,-4.227014,0.127338
87966,87966.0,28.0,0.123655,0.083543,0.207198,4.646482,165.050598,160.404114,-3.239736,170.698532,-3.252898,4.132565,-3.996469,-4.626503,0.127483
87952,87952.0,28.0,0.120951,0.162980,0.283931,5.668611,161.622330,155.953720,-3.163805,162.911972,-3.252406,5.135420,-2.723379,-5.661331,0.127256
105197,105197.0,33.0,0.094444,0.102566,0.197010,5.339356,155.679749,150.340393,-3.155228,178.069275,-3.241058,5.497356,-4.388076,-5.328547,0.128754
87944,87944.0,28.0,0.145730,0.110045,0.255774,3.086662,165.097870,162.011215,-3.075609,156.789337,-3.237451,2.950878,-3.670204,-2.990322,0.127450
87953,87953.0,28.0,0.108757,0.069656,0.178413,4.943049,160.020874,155.077820,-3.203660,152.532532,-3.237336,3.638846,-4.544709,-4.929323,0.126319
87941,87941.0,28.0,0.147713,0.122562,0.270275,4.133932,164.184692,160.050766,-2.455491,171.547394,-3.234285,3.101808,-4.035998,-4.099337,0.127956


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
112169,112169.0,35.0,0.102311,0.069313,0.171624,4.570127,158.738190,154.168060,-1.976245,147.523636,-2.123034,4.216288,-4.880044,-4.550669,0.127585
86740,86740.0,27.0,0.119463,0.135748,0.255211,4.462988,161.244400,156.781418,-1.881773,148.250458,-1.681880,6.735212,-3.690361,-4.436737,0.133617
61865,61865.0,19.0,0.133931,0.098283,0.232214,2.997247,164.454315,161.457062,-2.744731,148.276352,-2.514240,2.851284,-4.357836,-2.902160,0.128246
95680,95680.0,30.0,0.083011,0.106205,0.189217,4.360874,167.829285,163.468414,-2.536501,148.277954,-2.354380,4.449472,-4.595208,-4.338254,0.128062
17306,17306.0,5.0,0.100723,0.103565,0.204288,5.165435,165.118256,159.952820,-0.657419,148.365982,-1.503075,5.416565,-5.034744,-5.150604,0.127014
93619,93619.0,29.0,0.063982,0.097077,0.161059,4.896082,160.343872,155.447784,-2.385985,148.622055,-2.013040,5.080813,-4.454175,-4.880614,0.126317
60583,60583.0,19.0,0.118802,0.114613,0.233414,8.606842,166.342056,157.735214,-2.664401,148.666351,-2.425202,4.894450,-6.681119,-8.606441,0.124960
124770,124770.0,39.0,0.107109,0.088200,0.195309,5.508697,157.672379,152.163681,-2.344294,148.704285,-2.024565,3.358091,-6.127380,-5.501009,0.128141


## Read stored chi-squares for images

In [20]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [21]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [22]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [23]:
df_sliced=w.result
# df_sliced

In [24]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.04017924061040519, 'chi_spec1': 0.766557264222986}
[0, 39]
{'chi_1': 0.149562882362729, 'chi_spec1': 2.081281343647513}
[0, 39]
{'chi_1': 0.04017924061040519, 'chi_spec1': 0.766557264222986}
[array([60320, 79050]), array([35970, 22010]), array([33550, 50810])]
[22010 33550 35970 50810 60320 79050]


In [25]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [26]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(6, 22)
[(7, 22010), (10, 33550), (11, 35970), (16, 50810), (19, 60320), (25, 79050)]


In [27]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
2200,7-22010,0.017243,0.639168,32.881402,0.013206,0.003029,0.001008,0.006335,17.136884,42.257454,7,22010
3354,10-33550,0.019925,0.057654,30.565896,0.011231,0.006957,0.001736,0.010656,16.115353,3.565916,10,33550
3596,11-35970,0.086359,0.692672,33.975999,0.042608,0.042866,0.000884,0.009388,15.126607,16.206666,11,35970
5080,16-50810,0.014124,0.057967,29.795383,0.006589,0.002920,0.004615,0.003861,16.217060,2.702166,16,50810
6031,19-60320,0.004543,0.168560,30.094507,0.000814,0.001128,0.002601,0.000478,15.228170,9.508470,19,60320
7904,25-79050,0.004708,0.340449,29.074448,0.001671,0.001149,0.001889,0.000584,12.266147,25.771412,25,79050


### Plots 2: stored data

In [28]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [29]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')



In [30]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('7-22010', '10-33550', '11-35970', '1…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [31]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))



(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …